# Classifying images
In this tutorial you will learn how deep learning for image classification works and experiment with a very small convolutional neural network.

## Preliminaries

First, we'll import the libraries we need.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
# from PIL import Image

import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import initializers

# Keras utilities
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.callbacks import EarlyStopping
#from tensorflow.keras.models import Model
#from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
#from tensorflow.keras.optimizers import Adam




Now we'll import some utilities for visualisation etc. If you want to take a look at this code, uncomment the second line below before running it.

In [ ]:
from files import utils
# %load files/utils

## A simple model
The code in this section creates a very simple convolutional neural network that we can play around with.

In [ ]:
"""
Simple example network for visualising the filters and layers.
"""

num_classes = 2
image_size = 8

num_filters = 8 # also 4, 16
num_filters_l2 = 16
filter_size = 3 # also 4,5
# num_hidden = 64 # also 16?)

# batch_size = 5 # Half the images sent in each mini-batch
#batch_size = 10 # All images in each batch
batch_size = 1 # One at a time
# batch_size = 2 # 2 at a time

# initializer = initializers.Constant(0.1)
kernel_init = initializers.RandomUniform(minval=-0.01, maxval=0.01)

EPOCHS = 100

"""
data_augmentation = Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomFlip("vertical"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.5, interpolation='nearest', fill_mode='constant', fill_value=0.0),
    ]
)
"""

model = Sequential(
    [
        layers.Input((image_size, image_size, 3)),
        # data_augmentation,
        layers.Rescaling(1.0 / 255),                       # Rescale to 0..1
        layers.Conv2D(num_filters, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init),
        layers.MaxPooling2D(), # Halve the resolution to 4x4
        layers.Conv2D(num_filters_l2, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init),
        layers.MaxPooling2D(), # Halve the resolution to 2x2
        # layers.Conv2D(num_filters, filter_size, padding="same", activation="relu", kernel_initializer=kernel_init),
        # layers.MaxPooling2D(),
        # layers.Dropout(0.2),
        layers.Flatten(),                                                                       # Flatten filters into a 1D vector of vars
        ##layers.Dense(num_hidden, activation="relu"),                                            # Hidden layer
        layers.Dense(num_classes, activation="softmax", kernel_initializer=kernel_init),                                        # output layer
    ]
)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.save_weights('files/model.weights.h5') # Save the initialised weights so we can reset them later


## Visualising the model
We can get an overview of the structure of thje model using model.summary().

In [ ]:
model.summary(line_length=80)

The model (network) has now been created, but it hasn't been trained yet; the convolutional filter weights have been initialised to small random numbers. Let's see what they look like:

In [ ]:
utils.report_weights(model)

## training the model
Let's try training our model. We'll create two dataset generators to process the images into tensors and pass them into the model.
The model.fit(...) call then trains the model. The 'epochs' parameter controls how much data is fed into the model during training. We'll start with a short training run.
Note: this is a very small model, and the dataset is also small (both the image size and number of images). Normally we would expect training to take much longer, and would use a GPU to greatly speed up the process.

In [ ]:
# Do some training

# data_dir = "images_noisy_RGB_train"
# train_data_dir = "images_clean/train"  # Clean training data
# train_data_dir = "images_noisy_GB_train/train"  # Random FP in training set
data_dir = "files/images_red"  # Single-channel (red) images

train_data_dir = data_dir + "/train"
valid_data_dir = data_dir + "/val"
test_data_dir = data_dir + "/test"

train_generator = tf.keras.utils.image_dataset_from_directory(train_data_dir, image_size=(image_size,image_size), batch_size = batch_size)
valid_generator = tf.keras.utils.image_dataset_from_directory(valid_data_dir, image_size=(image_size,image_size), batch_size = batch_size)
    
# model.fit_generator(train_generator, epochs=fine_tune_epochs, validation_data=valid_generator, callbacks = callbacks)
model.fit(train_generator, validation_data=valid_generator, epochs=EPOCHS, shuffle=True)

Let's see how it does on some test images that were witheld from training:

In [ ]:
# Test the model
test_images = os.listdir(test_data_dir)
class_names = ['X', '0']
tot_correct = 0
for image_file in test_images:
    img = tf.keras.utils.load_img(os.path.join(test_data_dir, image_file), target_size=(image_size, image_size))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    predictions = model.predict(img_array)
    scores = tf.nn.softmax(predictions[0])
    score = np.max(scores)
    ans = class_names[np.argmax(scores)]
    correct = (ans == image_file[0]) # Filename is prefixed with class
    tot_correct += correct
    print(f"{image_file}: {ans} {score} {correct}")
print("==================================")
print(f"{tot_correct} of {len(test_images)} correct.")

What happened to the weights during training? Let's take a look.

In [ ]:
utils.report_weights(model)

The weights for the first layer of the network represent filters that extract low-level predictive features from the images, such as straight and curved lines. Our training data was monochrome (red), so for the green and blue channels the filters are (almost) empty. Unfortunately, visualising the next level of filters is less informative because they are (collectively) identifying patterns in the outputs from all of the filters in the previous layers. This information is highly distributed and difficult to decipher.

We can, however, also look at the *output* from the filters:



In [ ]:
# Example "X"
utils.report_outputs(model, os.path.join(test_data_dir, test_images[8]), image_size)

In [ ]:
# Example "O"
utils.report_outputs(model, os.path.join(test_data_dir, test_images[3]), image_size)

What are the filters 'looking at'? Each filter highlights the areas where the filter pattern matches, and dims the areas of the image where a match is poor. For example, the 'cross' image may have a particular diagonal highlighted, because this feature is found only in this class. For the 'nought' image, this same diagonal pattern picks up opposite 'corners' of the 'O' shape.

## Explaining the model - attribute importance




In [ ]:
utils.test_marginal_perm(test_data_dir, model, image_size)

## Noisy data
In the example above, the green and blue bands were uninformative because during training they were empty. Let's see what happens if we instead pass in noisy data. Let's see what it looks like compared to the "pure" red image used earlier.

<img src="files/images_red/train/nought/0_1.png" width="100"/>
<img src="files/images_noisy/train/nought/0_1.png" width="100"/>


The noisy images have random noise added to the green and blue channels - both false negatives and positives - but the red channel is still clean.

Let's see what happens when we retrain the model on this data.

First we'll recompile the model to reset the weights to random.

In [ ]:
model.load_weights('files/model.weights.h5', skip_mismatch=True)
utils.report_weights(model)

Now we'll retrain the model using the randomised imagery.

In [ ]:
# Train the model
data_dir = "files/images_noisy"

train_data_dir = data_dir + "/train"
valid_data_dir = data_dir + "/val"
test_data_dir = data_dir + "/test"

train_generator = tf.keras.utils.image_dataset_from_directory(train_data_dir, image_size=(image_size,image_size), batch_size = batch_size)
valid_generator = tf.keras.utils.image_dataset_from_directory(valid_data_dir, image_size=(image_size,image_size), batch_size = batch_size)
    
model.fit(train_generator, validation_data=valid_generator, epochs=EPOCHS, shuffle=True)

How did it do? Let's test it.

In [ ]:
# Test the model
print(test_data_dir)

test_images = os.listdir(test_data_dir)
class_names = ['X', '0']
tot_correct = 0
for image_file in test_images:
    img = tf.keras.utils.load_img(os.path.join(test_data_dir, image_file), target_size=(image_size, image_size))
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch
    predictions = model.predict(img_array)
    scores = tf.nn.softmax(predictions[0])
    score = np.max(scores)
    ans = class_names[np.argmax(scores)]
    correct = (ans == image_file[0]) # Filename is prefixed with class
    tot_correct += correct
    print(f"{image_file}: {ans} {score} {correct}")
print("==================================")
print(f"{tot_correct} of {len(test_images)} correct.")

What do the weights look like?

In [ ]:
utils.report_weights(model)

In [ ]:
utils.test_marginal_perm(test_data_dir, model, image_size)